In [3]:
import numpy as np
import pandas as pd

import os
import glob

In [4]:
#plotting things

#%matplotlib qt5 -- I don't know what this is
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from cycler import cycler


#All of Anandh's customized seaborn/matplotlib settings

sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 1.5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})

#%config InlineBackend.figure_f.ormats=['svg']

mpl.rc('axes', prop_cycle=(cycler('color', ['r', 'k', 'b','g','y','m','c']) ))

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

#mpl.rc('text', usetex=False)
#mpl.rc('text.latex', preamble=r'\usepackage{helvet}
#\renewcommand\familydefault{\sfdefault}\usepackage{sansmath}\sansmath')

    #If you want to use a different font
# mpl.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'], 
#                  'serif': ['Helvetica']})

tw = 1.5
sns.set_style({"xtick.major.size": 3, "ytick.major.size": 3,
               "xtick.minor.size": 2, "ytick.minor.size": 2,
               'axes.labelsize': 16, 'axes.titlesize': 16,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', linewidth=1.5)
mpl.rc('legend', fontsize=14)
mpl.rc('figure', figsize=(9,8))

## Get data

In [66]:
directory = 'Z:/Reed/Projects/micro_consortia/DARPA_biocon/Task 1.1/A=B/20181128 A=B cfp yfp screen 1/'

filename = '20181128 cfp yfp A=B screen 1.xlsx'

In [67]:
data_dict = pd.read_excel(directory + filename, sheet_name=None)

data_dict.keys()

odict_keys(['OD700', 'CFP61', 'CFP100', 'YFP61', 'YFP100', 'Exp', 'IDs', 'OD700_tidy', 'CFP61_tidy', 'CFP100_tidy', 'YFP61_tidy', 'YFP100_tidy'])

In [68]:
sheets_to_plot = [x for x in data_dict.keys() if '_tidy' in x]

In [94]:
ot, c61, c100, y61, y100 = [data_dict[sheet] for sheet in sheets_to_plot]

## Create normalized fluorescence

In [119]:
def join_fluor_and_od700 (od700_df, fluor_df):
    
    #get the time columns
    ft = np.sort(np.unique(fluor_df['Time']))
    ot = np.sort(np.unique(od700_df['Time']))
    
    #create dict to identify fluor time column entries with od time columns entries
    #every time you see time (cfp time entry), replace it with (od time entry)
    time_replacement_dict = {x : y for x,y in zip(ft,ot)}
    
    #add the time identifier column in the fluor df
    fluor_df['Time'] = fluor_df['Time'].map(time_replacement_dict)
    
    #merge the dfs, just merge the relevant entries in the fluor df though
    od700_df = od700_df.merge(fluor_df,
                              on=['Time', 'well'],
                              how='inner')
    
    #the merge adds duplicate columns with suffixes for columns that exist in both merge df's
    #this list removes the ones from the right (fluor_df) df
    cols_no_dup = [x for x in od700_df.columns if '_y' not in x]
    od700_df = od700_df.loc[:, cols_no_dup]
    
    #now get rid of the left suffixes in the remaining columns
    cols_no_suff = [x.strip('_x') for x in cols_no_dup]
    od700_df.columns = cols_no_suff
    
    return od700_df

In [120]:
#initial sheet to hold all normalization
od_norm_data = ot.copy()

In [121]:
all_fluor = [c61, c100, y61, y100]

#append all the fluorescence readings
for data in all_fluor:
    od_norm_data = join_fluor_and_od700(od_norm_data, data)

#some of the values have overflowed, replace them with highest recordable number
od_norm_data = od_norm_data.replace('OVRFLW', 99999)

In [124]:
fluor_cols = [c for c in od_norm_data.columns if 'fp' in c]

for fluor_col in fluor_cols:
    div = od_norm_data[fluor_col] / od_norm_data['od700']
    
    od_norm_data[fluor_col + '_odnorm'] = div

In [126]:
od_norm_data.to_csv(directory + 'od_norm_fluor.csv')